In [1]:
import datetime
import pandas as pd
import utils

log_path = "evaluator_output/evaluator_log.txt"

datetime_str = datetime.datetime.now().isoformat(sep=" ", timespec="seconds")
utils.log("Experiment " + datetime_str, log_path)

validate_ids = [3, 11, 39, 43]
#validate_ids = [x for x in range(48) if x not in example_ids and x not in range(9, 24)]
utils.log("validation data ids:" + str(validate_ids), log_path)

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('data/evaluator_filtered.csv')
display(df.iloc[validate_ids])

Experiment 2023-11-25 21:28:43
validation data ids:[3, 11, 39, 43]


,Text,Number of Tokens,Rating out of 3,Rating,Gan's Rating,Yimin's Rating
3,"Q: What is the difference between the Constitutions of the USA and USSR? Both guarantee freedom of speech. A: Yes, and the US Constitution also guarantees freedom after speech.",36,3,5,5,4
11,Q: What to do if a man you don't know takes a seat at your table in a pub and starts to sigh? A: Immediately demand to stop the anti-Soviet propaganda.,39,2,4,4,4
39,"Q: What is Sholokhov writing now? A: A crime novel titled ""How I Received the Nobel Prize in Literature.""",28,1,2,2,1
43,Q: When did kennedy get elected? A: John F. Kennedy 1961 presidential inauguration.,21,0,0,0,0


In [2]:
from evaluator import Evaluator
from sklearn.metrics import classification_report, mean_squared_error

evaluator = Evaluator(verbose=True, log_path=log_path)

ratings = []
explanations = []
for i in validate_ids:
    rating, explanation = evaluator.evaluate(df.iloc[i,0], verbose=True)
    ratings.append(rating)
    explanations.append(explanation)

df2 = df.iloc[validate_ids]
df2.insert(3, 'GPT Rating', ratings)
df2.insert(4, 'GPT Explanation', explanations)
df2.to_csv("evaluator_output/" + datetime_str.replace(" ", "_").replace(":", "-") + ".csv", index=False)

utils.log("rmse: " + str(mean_squared_error(df2.iloc[:,2], df2.iloc[:,3], squared=False)), log_path)
utils.log(classification_report(df2.iloc[:,2], df2.iloc[:,3], labels=range(4), zero_division=0.0), log_path)
utils.log("\n\n", log_path)

model: gpt-4-1106-preview, temperature: 1, top_p: 1
example data ids:[2, 10, 38, 42]
system prompt:
### instruction ###
Act as a Russian political joke evaluator. Evaluate the funniness with a reason and give an integer rating from 0 to 3, in a format of reason -> rating.
### examples ###
Example #1
<user>: '''Q: Is it true that every Soviet soldier dreams of becoming a general? A: No, our soldiers are not that stupid. They know that generals may become MIA even in time of peace.'''
<assistant>: '''This joke is very funny because it is easy to understand and plays on the absurdities and contradictions of the Soviet regime and its leadership -> 3'''
Example #2
<user>: '''Q: Could an atomic bomb destroy our beloved town, Yerevan, with its splendid buildings and beautiful gardens? A: In principle, yes. But Moscow is by far a more beautiful city.'''
<assistant>: '''This joke can be rated between 3 and 1 -> 2'''
Example #3
<user>: '''Q: Is there a difference between capitalism and communism